In [1]:
import openmc
import openmc.deplete
import numpy as np

In [2]:
#this is a model of a BCC lattice unit cell with a reflecting boundary

In [3]:
#materials
#these are functionally dummy values until you have a chance to look everything up.  note to self - include sources in comments.
#also doublecheck isotopic compostions conventions in mats
#graphite based on a3-3, triso layers pulled from reported values in Neutronics characteristics of a 165 MWth Xe-100 reactor, Mulder et al
uco = openmc.Material(name='UCO')
uco.set_density('g/cm3', 10.4)
uco.add_nuclide("U235", 0.1386, percent_type='wo')
uco.add_nuclide("U238",0.7559, percent_type='wo')
uco.add_element("O", 0.06025, percent_type='wo')
uco.add_element('C', 0.04523, percent_type='wo')
uco.add_s_alpha_beta('c_Graphite')
uco.temperature = 1159.15 #K

buffer = openmc.Material(name='buffer')
buffer.set_density('g/cm3', 1.05)
buffer.add_element('C', 1.0, percent_type='ao')
buffer.add_s_alpha_beta('c_Graphite')
buffer.temperature = 1159.15 #K

pyc = openmc.Material(name='PyC')
pyc.set_density('g/cm3', 1.9)
pyc.add_element('C', 1.0, percent_type='ao')
pyc.add_s_alpha_beta('c_Graphite')
u.temperature = 1159.15 #K

sic = openmc.Material(name='SiC')
sic.set_density('g/cm3', 3.2)
sic.add_element('C', 0.5, percent_type='ao')
sic.add_element('Si', 0.5, percent_type='ao')
sic.add_s_alpha_beta('c_Graphite')

graphite = openmc.Material(name='graphite')
graphite.set_density('kg/m3', 1700)
graphite.add_element('C', 1.0, percent_type='ao')
graphite.add_s_alpha_beta('c_Graphite')

he = openmc.Material(name='He')
he.set_density('atom/b-cm', 0.0006)
he.add_element('He', 1.0, percent_type='ao')
he.temperature = 778.15 #K

In [4]:
#geometry parameters - remember LAMMPS coords are in meters, this is in cm
#remember, triso order: kernel, buffer, pyc, sic, pyc
peb_or = 3.0 #outer radius of whole pebble
peb_ir = 2.5 # radius of the region that has trisos in it only
triso_r = [0.02125, 0.03125, 0.03525, 0.03875, 0.04275]
bcc_l = (4*peb_or)/(3**(0.5))
c_coord = 0.5*bcc_l

In [5]:
#define the edges of the unit cell
bcc_min_x = openmc.XPlane(x0=-c_coord, boundary_type='reflective')
bcc_max_x = openmc.XPlane(x0=c_coord, boundary_type='reflective')
bcc_min_y = openmc.YPlane(y0=-c_coord, boundary_type='reflective')
bcc_max_y = openmc.YPlane(y0=c_coord, boundary_type='reflective')
bcc_min_z = openmc.ZPlane(z0=-c_coord, boundary_type='reflective')
bcc_max_z = openmc.ZPlane(z0=c_coord, boundary_type='reflective')

In [6]:
#define the cells and universe for the triso particles
triso_bounds = [openmc.Sphere(r=r) for r in triso_r[:-1]]
triso_cells = [openmc.Cell(fill=uco, region=-triso_bounds[0]),
               openmc.Cell(fill=buffer, region=+triso_bounds[0] & -triso_bounds[1]),
               openmc.Cell(fill=pyc, region=+triso_bounds[1] & -triso_bounds[2]),
               openmc.Cell(fill=sic, region=+triso_bounds[2] & -triso_bounds[3]),
               openmc.Cell(fill=pyc, region=+triso_bounds[3])]
triso_univ = openmc.Universe(cells=triso_cells)

In [7]:
# define where the particles will be packed
# BCC lattice has one sphere at the center, and a 1/4 sphere at each corner, tight enough they are all touching.
#that part should be handled with your calculation of the cube length (the body diameter = 2 peb diameters)

#start with the easy part: the center pebble

body_peb_in = openmc.Sphere(r = peb_ir)
body_wfuel_bound = -body_peb_in
body_peb_out = openmc.Sphere(r=peb_or)
body_nofuel_reg = -body_peb_out & +body_peb_in

#remember, this is the fueled part only, so it uses peb_ir.  Center the BCC at 0.
#generate the triso centers in the body pebble:

body_centers = openmc.model.pack_spheres(triso_r[-1], region=body_wfuel_bound, num_spheres=19000, seed = 978397880)
body_trisos = [openmc.model.TRISO(triso_r[-1], triso_univ, center) for center in body_centers]

#now define the no-fueled outer shell for the body peb

body_wfuel = openmc.Cell(region=body_wfuel_bound)
lower_left, upper_right = body_wfuel.region.bounding_box
shape = (10, 10, 10)
pitch = (upper_right - lower_left)/shape
body_lattice = openmc.model.create_triso_lattice(
    body_trisos, lower_left, pitch, shape, graphite)

body_wfuel.fill = body_lattice
body_nofuel = openmc.Cell(fill=graphite, region=body_nofuel_reg)

body_cells = [body_wfuel, body_nofuel]


In [8]:
#this will follow the same basic steps as the center pebble.  initial triso definition shouldn't change.
#this one is the -1,-1,-1 corner
c1_peb_in = openmc.Sphere(x0 = -c_coord, y0 =-c_coord, z0 =-c_coord, r = peb_ir)
c1_peb_out = openmc.Sphere(x0 = -c_coord, y0 =-c_coord, z0 =-c_coord,r = peb_or)
c1_corner_bound = +bcc_min_x & +bcc_min_y & +bcc_min_z
c1_wfuel_reg = -c1_peb_in & c1_corner_bound
c1_nofuel_reg = -c1_peb_out & +c1_peb_in & c1_corner_bound

c1_centers = openmc.model.pack_spheres(triso_r[-1], region=-c1_peb_in, num_spheres=19000, seed = 987432789)
c1_trisos = [openmc.model.TRISO(triso_r[-1], triso_univ, center) for center in c1_centers]

c1_wfuel = openmc.Cell(region=c1_wfuel_reg)
lower_left, upper_right = c1_wfuel.region.bounding_box
shape = (5, 5, 5)
pitch = (upper_right - lower_left)/shape
c1_lattice = openmc.model.create_triso_lattice(
    c1_trisos, lower_left, pitch, shape, graphite)

c1_wfuel.fill = c1_lattice
c1_nofuel = openmc.Cell(fill=graphite, region=c1_nofuel_reg)

c1_cells = [c1_wfuel, c1_nofuel]

/home/zoe/anaconda3/envs/ghastly-dev/lib/python3.11/site-packages/openmc/model/triso.py:850: UserWarning: TRISO particle is partially or completely outside of the lattice.
  warnings.warn('TRISO particle is partially or completely '


In [9]:
#c2: -1,-1,1
c2_peb_in = openmc.Sphere(x0 = -c_coord, y0 =-c_coord, z0 =c_coord, r = peb_ir)
c2_peb_out = openmc.Sphere(x0 = -c_coord, y0 =-c_coord, z0 =c_coord,r = peb_or)
c2_corner_bound = +bcc_min_x & +bcc_min_y & -bcc_max_z
c2_wfuel_reg = -c2_peb_in & c2_corner_bound
c2_nofuel_reg = -c2_peb_out & +c2_peb_in & c2_corner_bound

c2_centers = openmc.model.pack_spheres(triso_r[-1], region=-c2_peb_in, num_spheres=19000, seed = 895490889)
c2_trisos = [openmc.model.TRISO(triso_r[-1], triso_univ, center) for center in c2_centers]

c2_wfuel = openmc.Cell(region=c2_wfuel_reg)
lower_left, upper_right = c2_wfuel.region.bounding_box
shape = (5, 5, 5)
pitch = (upper_right - lower_left)/shape
c2_lattice = openmc.model.create_triso_lattice(
    c2_trisos, lower_left, pitch, shape, graphite)

c2_wfuel.fill = c2_lattice
c2_nofuel = openmc.Cell(fill=graphite, region=c2_nofuel_reg)

c2_cells = [c2_wfuel, c2_nofuel]

In [10]:
#c3: -1, 1, -1
c3_peb_in = openmc.Sphere(x0 = -c_coord, y0 =c_coord, z0 =-c_coord, r = peb_ir)
c3_peb_out = openmc.Sphere(x0 = -c_coord, y0 =c_coord, z0 =-c_coord,r = peb_or)
c3_corner_bound = +bcc_min_x & -bcc_max_y & +bcc_min_z
c3_wfuel_reg = -c3_peb_in & c3_corner_bound
c3_nofuel_reg = -c3_peb_out & +c3_peb_in & c3_corner_bound

c3_centers = openmc.model.pack_spheres(triso_r[-1], region=-c3_peb_in, num_spheres=19000, seed = 356657913)
c3_trisos = [openmc.model.TRISO(triso_r[-1], triso_univ, center) for center in c3_centers]

c3_wfuel = openmc.Cell(region=c3_wfuel_reg)
lower_left, upper_right = c3_wfuel.region.bounding_box
shape = (5, 5, 5)
pitch = (upper_right - lower_left)/shape
c3_lattice = openmc.model.create_triso_lattice(
    c3_trisos, lower_left, pitch, shape, graphite)

c3_wfuel.fill = c3_lattice
c3_nofuel = openmc.Cell(fill=graphite, region=c3_nofuel_reg)

c3_cells = [c3_wfuel, c3_nofuel]

In [11]:
#c4: -1, 1, 1
c4_peb_in = openmc.Sphere(x0 = -c_coord, y0 =c_coord, z0 =c_coord, r = peb_ir)
c4_peb_out = openmc.Sphere(x0 = -c_coord, y0 =c_coord, z0 =c_coord,r = peb_or)
c4_corner_bound = +bcc_min_x & -bcc_max_y & -bcc_max_z
c4_wfuel_reg = -c4_peb_in & c4_corner_bound
c4_nofuel_reg = -c4_peb_out & +c4_peb_in & c4_corner_bound


c4_centers = openmc.model.pack_spheres(triso_r[-1], region=-c4_peb_in, num_spheres=19000, seed = 353623684)
c4_trisos = [openmc.model.TRISO(triso_r[-1], triso_univ, center) for center in c4_centers]

c4_wfuel = openmc.Cell(region=c4_wfuel_reg)
lower_left, upper_right = c4_wfuel.region.bounding_box
shape = (5, 5, 5)
pitch = (upper_right - lower_left)/shape
c4_lattice = openmc.model.create_triso_lattice(
    c4_trisos, lower_left, pitch, shape, graphite)

c4_wfuel.fill = c4_lattice
c4_nofuel = openmc.Cell(fill=graphite, region=c4_nofuel_reg)

c4_cells = [c4_wfuel, c4_nofuel]

In [12]:
#c5: 1, -1, -1
c5_peb_in = openmc.Sphere(x0 = c_coord, y0 =-c_coord, z0 =-c_coord, r = peb_ir)
c5_peb_out = openmc.Sphere(x0 = c_coord, y0 =-c_coord, z0 =-c_coord,r = peb_or)
c5_corner_bound = -bcc_max_x & +bcc_min_y & +bcc_min_z
c5_wfuel_reg = -c5_peb_in & c5_corner_bound
c5_nofuel_reg = -c5_peb_out & +c5_peb_in & c5_corner_bound

c5_centers = openmc.model.pack_spheres(triso_r[-1], region=-c5_peb_in, num_spheres=19000, seed = 897578459)
c5_trisos = [openmc.model.TRISO(triso_r[-1], triso_univ, center) for center in c5_centers]

c5_wfuel = openmc.Cell(region=c5_wfuel_reg)
lower_left, upper_right = c5_wfuel.region.bounding_box
shape = (5, 5, 5)
pitch = (upper_right - lower_left)/shape
c5_lattice = openmc.model.create_triso_lattice(
    c5_trisos, lower_left, pitch, shape, graphite)

c5_wfuel.fill = c5_lattice
c5_nofuel = openmc.Cell(fill=graphite, region=c5_nofuel_reg)

c5_cells = [c5_wfuel, c5_nofuel]

In [13]:
#c6: 1, -1, 1
c6_peb_in = openmc.Sphere(x0 = c_coord, y0 =-c_coord, z0 =c_coord, r = peb_ir)
c6_peb_out = openmc.Sphere(x0 = c_coord, y0 =-c_coord, z0 =c_coord,r = peb_or)
c6_corner_bound = -bcc_max_x & +bcc_min_y & -bcc_max_z
c6_wfuel_reg = -c6_peb_in & c6_corner_bound
c6_nofuel_reg = -c6_peb_out & +c6_peb_in & c6_corner_bound

c6_centers = openmc.model.pack_spheres(triso_r[-1], region=-c6_peb_in, num_spheres=19000, seed = 429875621)
c6_trisos = [openmc.model.TRISO(triso_r[-1], triso_univ, center) for center in c6_centers]

c6_wfuel = openmc.Cell(region=c6_wfuel_reg)
lower_left, upper_right = c6_wfuel.region.bounding_box
shape = (5, 5, 5)
pitch = (upper_right - lower_left)/shape
c6_lattice = openmc.model.create_triso_lattice(
    c6_trisos, lower_left, pitch, shape, graphite)

c6_wfuel.fill = c6_lattice
c6_nofuel = openmc.Cell(fill=graphite, region=c6_nofuel_reg)

c6_cells = [c6_wfuel, c6_nofuel]

In [14]:
#c7: 1, 1, -1
c7_peb_in = openmc.Sphere(x0 = c_coord, y0 =c_coord, z0 =-c_coord, r = peb_ir)
c7_peb_out = openmc.Sphere(x0 = c_coord, y0 =c_coord, z0 =-c_coord,r = peb_or)
c7_corner_bound = -bcc_max_x & -bcc_max_y & +bcc_min_z
c7_wfuel_reg = -c7_peb_in & c7_corner_bound
c7_nofuel_reg = -c7_peb_out & +c7_peb_in & c7_corner_bound

c7_centers = openmc.model.pack_spheres(triso_r[-1], region=-c7_peb_in, num_spheres=19000, seed = 489638795)
c7_trisos = [openmc.model.TRISO(triso_r[-1], triso_univ, center) for center in c7_centers]

c7_wfuel = openmc.Cell(region=c7_wfuel_reg)
lower_left, upper_right = c7_wfuel.region.bounding_box
shape = (5, 5, 5)
pitch = (upper_right - lower_left)/shape
c7_lattice = openmc.model.create_triso_lattice(
    c7_trisos, lower_left, pitch, shape, graphite)

c7_wfuel.fill = c7_lattice
c7_nofuel = openmc.Cell(fill=graphite, region=c7_nofuel_reg)

c7_cells = [c7_wfuel, c7_nofuel]

In [15]:
#c8: 1, 1, 1
c8_peb_in = openmc.Sphere(x0 = c_coord, y0 =c_coord, z0 =c_coord, r = peb_ir)
c8_peb_out = openmc.Sphere(x0 = c_coord, y0 =c_coord, z0 =c_coord,r = peb_or)
c8_corner_bound = -bcc_max_x & -bcc_max_y & -bcc_max_z
c8_wfuel_reg = -c8_peb_in & c8_corner_bound
c8_nofuel_reg = -c8_peb_out & +c8_peb_in & c8_corner_bound

c8_centers = openmc.model.pack_spheres(triso_r[-1], region=-c8_peb_in, num_spheres=19000, seed = 658469875)
c8_trisos = [openmc.model.TRISO(triso_r[-1], triso_univ, center) for center in c8_centers]

c8_wfuel = openmc.Cell(region=c8_wfuel_reg)
lower_left, upper_right = c8_wfuel.region.bounding_box
shape = (5, 5, 5)
pitch = (upper_right - lower_left)/shape
c8_lattice = openmc.model.create_triso_lattice(
    c8_trisos, lower_left, pitch, shape, graphite)

c8_wfuel.fill = c8_lattice
c8_nofuel = openmc.Cell(fill=graphite, region=c8_nofuel_reg)

c8_cells = [c8_wfuel, c8_nofuel]

In [16]:
bcc_out_bounds = +bcc_min_x & -bcc_max_x & +bcc_min_y & -bcc_max_y & +bcc_min_z & -bcc_max_z
bcc_fill_reg = (bcc_out_bounds & +body_peb_out & +c1_peb_out & +c2_peb_out 
                & +c3_peb_out & +c4_peb_out & +c5_peb_out & +c6_peb_out & +c7_peb_out & +c8_peb_out)
bcc_fill = openmc.Cell(region=bcc_fill_reg, fill=he)

In [17]:
cell_list = body_cells + c1_cells + c2_cells + c3_cells + c4_cells + c5_cells + c6_cells + c7_cells + c8_cells + [bcc_fill]
universe = openmc.Universe(cells=cell_list)

geometry = openmc.Geometry(universe)
geometry.export_to_xml()

#materials = list(geometry.get_all_materials().values())
#openmc.Materials(materials).export_to_xml()

materials = openmc.Materials([uco, buffer, pyc, sic, graphite, he])
openmc.Materials(materials).export_to_xml()

# in order to accurately deplete the unit cell, we need the total volume of uco inside it
#because the corners are randomly packed and then cut off, you can't just use the total volume of all triso kernels
#instead, run a stochastic volume calc for uco, then use that
lower_left = (-c_coord, -c_coord, -c_coord)
upper_right = (c_coord, c_coord, c_coord)
vol_calc = openmc.VolumeCalculation([uco], 1000000,
                                    lower_left, upper_right)
vol_calc.set_trigger(0.005, 'rel_err')

settings = openmc.Settings()
#settings.run_mode = 'eigenvalue'
settings.particles = 25000
settings.verbosity = 5
settings.batches = 50
settings.inactive = 50
#settings.volume_calculations = [vol_calc]
settings.export_to_xml()




In [18]:
bcc_model = openmc.model.Model(geometry, materials, settings)

In [19]:

#openmc.calculate_volumes()

In [19]:
#chain file set as environment variable
uco_volume = 1.5209
uco.volume = uco_volume
bcc_operator = openmc.deplete.CoupledOperator(bcc_model)
# 1549 effective full power days over 6 passes - 51 30 day steps = 1530, 52(30) = 1560, so go to 52
deplete_steps = [30*24*60*60]*52

In [20]:
reactor_power = 165.0*(10**6) #165 MWth, converted to W
#220K pebs, 19k triso per peb, vol_kernel, uco density, wt percent of u in uco
reactor_uco_weight = 220000*19000*(4/3)*np.pi*triso_r[0]**3*uco.density*0.8945
specific_power = reactor_power/reactor_uco_weight #this is in W/gHM

In [ ]:
celi = openmc.deplete.CELIIntegrator(

    bcc_operator, deplete_steps, power_density=specific_power)

celi.integrate()

                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%%%%%%%%%%%%%%%%%
                #####################     %%%%%%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%%%%%
                 #######################     %%%%%%%%%%%%%%%%%
                 #####################

 Reading Na23 from /home/zoe/xs/endfb80/endfb-viii.0-hdf5/neutron/Na23.h5
 Reading Mg24 from /home/zoe/xs/endfb80/endfb-viii.0-hdf5/neutron/Mg24.h5
 Reading Mg25 from /home/zoe/xs/endfb80/endfb-viii.0-hdf5/neutron/Mg25.h5
 Reading Mg26 from /home/zoe/xs/endfb80/endfb-viii.0-hdf5/neutron/Mg26.h5
 Reading Al26_m1 from /home/zoe/xs/endfb80/endfb-viii.0-hdf5/neutron/Al26_m1.h5
 Reading Al27 from /home/zoe/xs/endfb80/endfb-viii.0-hdf5/neutron/Al27.h5
 Reading Si31 from /home/zoe/xs/endfb80/endfb-viii.0-hdf5/neutron/Si31.h5
 Reading Si32 from /home/zoe/xs/endfb80/endfb-viii.0-hdf5/neutron/Si32.h5
 Reading P31 from /home/zoe/xs/endfb80/endfb-viii.0-hdf5/neutron/P31.h5
 Reading S32 from /home/zoe/xs/endfb80/endfb-viii.0-hdf5/neutron/S32.h5
 Reading S33 from /home/zoe/xs/endfb80/endfb-viii.0-hdf5/neutron/S33.h5
 Reading S34 from /home/zoe/xs/endfb80/endfb-viii.0-hdf5/neutron/S34.h5
 Reading S35 from /home/zoe/xs/endfb80/endfb-viii.0-hdf5/neutron/S35.h5
 Reading S36 from /home/zoe/xs/endfb80/end

 Reading Ar38 from /home/zoe/xs/endfb80/endfb-viii.0-hdf5/neutron/Ar38.h5
 Reading Ar39 from /home/zoe/xs/endfb80/endfb-viii.0-hdf5/neutron/Ar39.h5
 Reading Ar40 from /home/zoe/xs/endfb80/endfb-viii.0-hdf5/neutron/Ar40.h5
 Reading Ar41 from /home/zoe/xs/endfb80/endfb-viii.0-hdf5/neutron/Ar41.h5
 Reading K39 from /home/zoe/xs/endfb80/endfb-viii.0-hdf5/neutron/K39.h5
 Reading K40 from /home/zoe/xs/endfb80/endfb-viii.0-hdf5/neutron/K40.h5
 Reading K41 from /home/zoe/xs/endfb80/endfb-viii.0-hdf5/neutron/K41.h5
 Reading Ca40 from /home/zoe/xs/endfb80/endfb-viii.0-hdf5/neutron/Ca40.h5
 Reading Ca41 from /home/zoe/xs/endfb80/endfb-viii.0-hdf5/neutron/Ca41.h5
 Reading Ca42 from /home/zoe/xs/endfb80/endfb-viii.0-hdf5/neutron/Ca42.h5
 Reading Ca43 from /home/zoe/xs/endfb80/endfb-viii.0-hdf5/neutron/Ca43.h5
 Reading Ca44 from /home/zoe/xs/endfb80/endfb-viii.0-hdf5/neutron/Ca44.h5
 Reading Ca45 from /home/zoe/xs/endfb80/endfb-viii.0-hdf5/neutron/Ca45.h5
 Reading Ca46 from /home/zoe/xs/endfb80/endf

 Reading Se80 from /home/zoe/xs/endfb80/endfb-viii.0-hdf5/neutron/Se80.h5
 Reading Se81 from /home/zoe/xs/endfb80/endfb-viii.0-hdf5/neutron/Se81.h5
 Reading Se82 from /home/zoe/xs/endfb80/endfb-viii.0-hdf5/neutron/Se82.h5
 Reading Br79 from /home/zoe/xs/endfb80/endfb-viii.0-hdf5/neutron/Br79.h5
 Reading Br80 from /home/zoe/xs/endfb80/endfb-viii.0-hdf5/neutron/Br80.h5
 Reading Br81 from /home/zoe/xs/endfb80/endfb-viii.0-hdf5/neutron/Br81.h5
 Reading Kr78 from /home/zoe/xs/endfb80/endfb-viii.0-hdf5/neutron/Kr78.h5
 Reading Kr79 from /home/zoe/xs/endfb80/endfb-viii.0-hdf5/neutron/Kr79.h5
 Reading Kr80 from /home/zoe/xs/endfb80/endfb-viii.0-hdf5/neutron/Kr80.h5
 Reading Kr81 from /home/zoe/xs/endfb80/endfb-viii.0-hdf5/neutron/Kr81.h5
 Reading Kr82 from /home/zoe/xs/endfb80/endfb-viii.0-hdf5/neutron/Kr82.h5
 Reading Kr83 from /home/zoe/xs/endfb80/endfb-viii.0-hdf5/neutron/Kr83.h5
 Reading Kr84 from /home/zoe/xs/endfb80/endfb-viii.0-hdf5/neutron/Kr84.h5
 Reading Kr85 from /home/zoe/xs/endfb8

 Reading Nb94 from /home/zoe/xs/endfb80/endfb-viii.0-hdf5/neutron/Nb94.h5
 Reading Nb95 from /home/zoe/xs/endfb80/endfb-viii.0-hdf5/neutron/Nb95.h5


...

 Reading Hf174 from /home/zoe/xs/endfb80/endfb-viii.0-hdf5/neutron/Hf174.h5
 Reading Hf175 from /home/zoe/xs/endfb80/endfb-viii.0-hdf5/neutron/Hf175.h5
 Reading Hf176 from /home/zoe/xs/endfb80/endfb-viii.0-hdf5/neutron/Hf176.h5
 Reading Hf177 from /home/zoe/xs/endfb80/endfb-viii.0-hdf5/neutron/Hf177.h5
 Reading Hf178 from /home/zoe/xs/endfb80/endfb-viii.0-hdf5/neutron/Hf178.h5
 Reading Hf179 from /home/zoe/xs/endfb80/endfb-viii.0-hdf5/neutron/Hf179.h5
 Reading Hf180 from /home/zoe/xs/endfb80/endfb-viii.0-hdf5/neutron/Hf180.h5
 Reading Hf181 from /home/zoe/xs/endfb80/endfb-viii.0-hdf5/neutron/Hf181.h5
 Reading Hf182 from /home/zoe/xs/endfb80/endfb-viii.0-hdf5/neutron/Hf182.h5
 Reading Ta180 from /home/zoe/xs/endfb80/endfb-viii.0-hdf5/neutron/Ta180.h5


          1200K
          2500K
          1200K
          2500K


 Reading Ta181 from /home/zoe/xs/endfb80/endfb-viii.0-hdf5/neutron/Ta181.h5
 Reading Ta182 from /home/zoe/xs/endfb80/endfb-viii.0-hdf5/neutron/Ta182.h5
 Reading W180 from /home/zoe/xs/endfb80/endfb-viii.0-hdf5/neutron/W180.h5
 Reading W181 from /home/zoe/xs/endfb80/endfb-viii.0-hdf5/neutron/W181.h5
 Reading W182 from /home/zoe/xs/endfb80/endfb-viii.0-hdf5/neutron/W182.h5
 Reading W183 from /home/zoe/xs/endfb80/endfb-viii.0-hdf5/neutron/W183.h5
 Reading W184 from /home/zoe/xs/endfb80/endfb-viii.0-hdf5/neutron/W184.h5
 Reading W185 from /home/zoe/xs/endfb80/endfb-viii.0-hdf5/neutron/W185.h5
 Reading W186 from /home/zoe/xs/endfb80/endfb-viii.0-hdf5/neutron/W186.h5
 Reading Re185 from /home/zoe/xs/endfb80/endfb-viii.0-hdf5/neutron/Re185.h5
 Reading Re186_m1 from
 /home/zoe/xs/endfb80/endfb-viii.0-hdf5/neutron/Re186_m1.h5
 Reading Re187 from /home/zoe/xs/endfb80/endfb-viii.0-hdf5/neutron/Re187.h5
 Reading Os184 from /home/zoe/xs/endfb80/endfb-viii.0-hdf5/neutron/Os184.h5
 Reading Os185 from

          1200K


 Reading Cf253 from /home/zoe/xs/endfb80/endfb-viii.0-hdf5/neutron/Cf253.h5
 Reading Cf254 from /home/zoe/xs/endfb80/endfb-viii.0-hdf5/neutron/Cf254.h5
 Reading Es251 from /home/zoe/xs/endfb80/endfb-viii.0-hdf5/neutron/Es251.h5
 Reading Es252 from /home/zoe/xs/endfb80/endfb-viii.0-hdf5/neutron/Es252.h5
 Reading Es253 from /home/zoe/xs/endfb80/endfb-viii.0-hdf5/neutron/Es253.h5
 Reading Es254 from /home/zoe/xs/endfb80/endfb-viii.0-hdf5/neutron/Es254.h5
 Reading Es254_m1 from
 /home/zoe/xs/endfb80/endfb-viii.0-hdf5/neutron/Es254_m1.h5
 Reading Es255 from /home/zoe/xs/endfb80/endfb-viii.0-hdf5/neutron/Es255.h5
 Reading Fm255 from /home/zoe/xs/endfb80/endfb-viii.0-hdf5/neutron/Fm255.h5
 Maximum neutron transport energy: 20000000 eV for C13
 Initializing source particles...

 ====================>     K EIGENVALUE SIMULATION     <====================

  Bat./Gen.      k            Average k
  =========   ========   ====================
        1/1    1.59123
        2/1    1.61657
        3

In [ ]:
plot_mat_zoom = openmc.Plot.from_geometry(geometry)
plot_mat_zoom.color_by = 'material'
plot_mat_zoom.colors = {graphite: 'lightgray', he: 'lavender', uco: 'midnightblue', pyc: 'darkseagreen', sic: 'seagreen', buffer:'honeydew'}
plot_mat_zoom.basis = 'xy'
plot_mat_zoom.origin = (0, 0, 0)
plot_mat_zoom.width = (0.25*bcc_l, 0.25*bcc_l)
plot_mat_zoom.pixels = (2000, 2000)
plot_mat_zoom.filename = 'mat_zoom.png'
openmc.plot_inline(plot_mat_zoom)

In [ ]:
plot_cell_body = openmc.Plot.from_geometry(geometry)
plot_cell_body.show_overlaps = True
plot_cell_body.overlap_color = 'red'
plot_cell_body.basis = 'xy'
plot_cell_body.origin = (0, 0, 0)
plot_cell_body.width = (bcc_l, bcc_l)
plot_cell_body.pixels = (2000, 2000)
plot_cell_body.filename = 'cell_body.png'
openmc.plot_inline(plot_cell_body)

In [ ]:
plot_mat_body = openmc.Plot.from_geometry(geometry)
plot_mat_body.color_by = 'material'
plot_mat_body.colors = {graphite: 'lightgray', he: 'lavender', uco: 'midnightblue', pyc: 'darkseagreen', sic: 'seagreen', buffer:'honeydew'}
plot_mat_body.basis = 'xy'
plot_mat_body.origin = (0, 0, 0)
plot_mat_body.width = (bcc_l, bcc_l)
plot_mat_body.pixels = (1000, 1000)
plot_mat_body.filename = 'mat_body.png'
openmc.plot_inline(plot_mat_body)

In [ ]:
plot_cell_p1 = openmc.Plot.from_geometry(geometry)
plot_cell_p1.show_overlaps = True
plot_cell_p1.overlap_color = 'red'
plot_cell_p1.basis = 'xy'
plot_cell_p1.origin = (0, 0, -0.4999*bcc_l)
plot_cell_p1.width = (bcc_l, bcc_l)
plot_cell_p1.pixels = (2000, 2000)
plot_cell_p1.filename = 'cell_0_0_-1.png'
openmc.plot_inline(plot_cell_p1)

In [ ]:
plot_mat_p1 = openmc.Plot.from_geometry(geometry)
plot_mat_p1.color_by = 'material'
plot_mat_p1.colors = {graphite: 'lightgray', he: 'lavender', uco: 'midnightblue', pyc: 'darkseagreen', sic: 'seagreen', buffer:'honeydew'}
plot_mat_p1.basis = 'xy'
plot_mat_p1.origin = (0, 0, -0.4999*bcc_l)
plot_mat_p1.width = (bcc_l, bcc_l)
plot_mat_p1.pixels = (2000, 2000)
plot_mat_p1.filename = 'mat_0_0_-1.png'
openmc.plot_inline(plot_mat_p1)

In [ ]:
plot_cell_p2 = openmc.Plot.from_geometry(geometry)
plot_cell_p2.show_overlaps = True
plot_cell_p2.overlap_color = 'red'
plot_cell_p2.basis = 'xy'
plot_cell_p2.origin = (0, 0, 0.4999*bcc_l)
plot_cell_p2.width = (bcc_l, bcc_l)
plot_cell_p2.pixels = (2000, 2000)
plot_cell_p2.filename = 'cell_0_0_1.png'
openmc.plot_inline(plot_cell_p2)

In [ ]:
plot_mat_p2 = openmc.Plot.from_geometry(geometry)
plot_mat_p2.color_by = 'material'
plot_mat_p2.colors = {graphite: 'lightgray', he: 'lavender', uco: 'midnightblue', pyc: 'darkseagreen', sic: 'seagreen', buffer:'honeydew'}
plot_mat_p2.basis = 'xy'
plot_mat_p2.origin = (0, 0, 0.4999*bcc_l)
plot_mat_p2.width = (bcc_l, bcc_l)
plot_mat_p2.pixels = (2000, 2000)
plot_mat_p2.filename = 'mat_0_0_1.png'
openmc.plot_inline(plot_mat_p2)

In [ ]:
plot_cell_p3 = openmc.Plot.from_geometry(geometry)
plot_cell_p3.show_overlaps = True
plot_cell_p3.overlap_color = 'red'
plot_cell_p3.basis = 'xz'
plot_cell_p3.origin = (0, -0.4999*bcc_l, 0)
plot_cell_p3.width = (bcc_l, bcc_l)
plot_cell_p3.pixels = (2000, 2000)
plot_cell_p3.filename = 'cell_0_-1_0.png'
openmc.plot_inline(plot_cell_p3)

In [ ]:
plot_mat_p3 = openmc.Plot.from_geometry(geometry)
plot_mat_p3.color_by = 'material'
plot_mat_p3.colors = {graphite: 'lightgray', he: 'lavender', uco: 'midnightblue', pyc: 'darkseagreen', sic: 'seagreen', buffer:'honeydew'}
plot_mat_p3.basis = 'xz'
plot_mat_p3.origin = (0, -0.4999*bcc_l, 0)
plot_mat_p3.width = (bcc_l, bcc_l)
plot_mat_p3.pixels = (2000, 2000)
plot_mat_p3.filename = 'mat_0_-1_0.png'
openmc.plot_inline(plot_mat_p3)

In [ ]:
plot_cell_p4 = openmc.Plot.from_geometry(geometry)
plot_cell_p4.show_overlaps = True
plot_cell_p4.overlap_color = 'red'
plot_cell_p4.basis = 'xz'
plot_cell_p4.origin = (0, 0.4999*bcc_l, 0)
plot_cell_p4.width = (bcc_l, bcc_l)
plot_cell_p4.pixels = (2000, 2000)
plot_cell_p4.filename = 'cell_0_1_0.png'
openmc.plot_inline(plot_cell_p4)

In [ ]:
plot_mat_p4 = openmc.Plot.from_geometry(geometry)
plot_mat_p4.color_by = 'material'
plot_mat_p4.colors = {graphite: 'lightgray', he: 'lavender', uco: 'midnightblue', pyc: 'darkseagreen', sic: 'seagreen', buffer:'honeydew'}
plot_mat_p4.basis = 'xz'
plot_mat_p4.origin = (0, 0.4999*bcc_l, 0)
plot_mat_p4.width = (bcc_l, bcc_l)
plot_mat_p4.pixels = (2000, 2000)
plot_mat_p4.filename = 'mat_0_1_0.png'
openmc.plot_inline(plot_mat_p4)

In [ ]:
plot_cell_p5 = openmc.Plot.from_geometry(geometry)
plot_cell_p5.show_overlaps = True
plot_cell_p5.overlap_color = 'red'
plot_cell_p5.basis = 'yz'
plot_cell_p5.origin = (-0.4999*bcc_l, 0, 0)
plot_cell_p5.width = (bcc_l, bcc_l)
plot_cell_p5.pixels = (2000, 2000)
plot_cell_p5.filename = 'cell_-1_0_0.png'
openmc.plot_inline(plot_cell_p5)

In [ ]:
plot_mat_p5 = openmc.Plot.from_geometry(geometry)
plot_mat_p5.color_by = 'material'
plot_mat_p5.colors = {graphite: 'lightgray', he: 'lavender', uco: 'midnightblue', pyc: 'darkseagreen', sic: 'seagreen', buffer:'honeydew'}
plot_mat_p5.basis = 'yz'
plot_mat_p5.origin = (-0.4999*bcc_l, 0, 0)
plot_mat_p5.width = (bcc_l, bcc_l)
plot_mat_p5.pixels = (2000, 2000)
plot_mat_p5.filename = 'mat_-1_0_0.png'
openmc.plot_inline(plot_mat_p5)

In [ ]:
plot_cell_p6 = openmc.Plot.from_geometry(geometry)
plot_cell_p6.show_overlaps = True
plot_cell_p6.overlap_color = 'red'
plot_cell_p6.basis = 'yz'
plot_cell_p6.origin = (0.4999*bcc_l, 0, 0)
plot_cell_p6.width = (bcc_l, bcc_l)
plot_cell_p6.pixels = (2000, 2000)
plot_cell_p6.filename = 'cell_1_0_0.png'
openmc.plot_inline(plot_cell_p6)

In [ ]:
plot_mat_p6 = openmc.Plot.from_geometry(geometry)
plot_mat_p6.color_by = 'material'
plot_mat_p6.colors = {graphite: 'lightgray', he: 'lavender', uco: 'midnightblue', pyc: 'darkseagreen', sic: 'seagreen', buffer:'honeydew'}
plot_mat_p6.basis = 'yz'
plot_mat_p6.origin = (0.4999*bcc_l, 0, 0)
plot_mat_p6.width = (bcc_l, bcc_l)
plot_mat_p6.pixels = (2000, 2000)
plot_mat_p6.filename = 'mat_1_0_0.png'
openmc.plot_inline(plot_mat_p6)

In [ ]:
vox_plot = openmc.Plot()
vox_plot.type = 'voxel'
vox_plot.color_by = 'cell'
vox_plot.origin = (0.0, 0.0, 0.0)
vox_plot.width = (bcc_l, bcc_l, bcc_l)
vox_plot.pixels = (500, 500, 500)
vox_plot.filename = 'bcc.h5'
vox_plots = openmc.Plots([vox_plot])
vox_plots.export_to_xml()
openmc.plot_geometry()

In [ ]:
#vox_plot.to_vtk(output='bcc.vti')
openmc.voxel_to_vtk('bcc.h5', output='bcc.vti')

In [ ]:
ray_plot = openmc.SolidRayTracePlot()
ray_plot.colors = {graphite: 'lightgray', he: 'lavender', uco: 'midnightblue', pyc: 'darkseagreen', sic: 'seagreen', buffer:'honeydew'}
ray_plot.pixels = (2000, 2000)
ray_plot.camera_position = (bcc_l, bcc_l, 0.75*bcc_l)
ray_plot.look_at = (0.0, 0.0, 0.0)
ray_plot.color_by = 'material'
ray_plot.light_position = (2*bcc_l, 3*bcc_l, 4*bcc_l)
ray_plot.diffuse_fraction = 0.50
ray_plot.opaque_domains = [graphite, uco, pyc, sic, buffer]
ray_plot.filename = '3d-bcc-mat.png'

In [ ]:
ray_plots = openmc.Plots([ray_plot])
ray_plots.export_to_xml()
openmc.plot_geometry()

In [ ]:
ray_plot2 = openmc.SolidRayTracePlot()
cell_color_dict = {body_wfuel:'slategray',
                  body_nofuel: 'silver',
                  c1_wfuel: 'purple',
                  c1_nofuel: 'plum',
                  c2_wfuel: 'royalblue',
                  c2_nofuel: 'skyblue',
                  c3_wfuel: 'green',
                  c3_nofuel: 'lightgreen',
                  c4_wfuel: 'hotpink',
                  c4_nofuel: 'pink',
                  c5_wfuel: 'gold',
                  c5_nofuel: 'palegoldenrod',
                  c6_wfuel: 'darkorange',
                  c6_nofuel: 'lightsalmon',
                  c7_wfuel: 'maroon',
                  c7_nofuel: 'lightcoral',
                  c8_wfuel: 'mediumvioletred',
                  c8_nofuel: 'palevioletred',
                  bcc_fill: 'lavender'}
ray_plot2.colors = cell_color_dict
ray_plot2.pixels = (2000, 2000)
ray_plot2.camera_position = (bcc_l, bcc_l, 0.75*bcc_l)
ray_plot2.look_at = (0.0, 0.0, 0.0)
ray_plot2.color_by = 'cell'
ray_plot2.light_position = (2*bcc_l, 3*bcc_l, 4*bcc_l)
ray_plot2.diffuse_fraction = 0.1
ray_plot2.opaque_domains = [body_wfuel, c1_wfuel, c2_wfuel, c3_wfuel, c4_wfuel, c5_wfuel, c6_wfuel, c7_wfuel, c8_wfuel,
                           body_nofuel, c1_nofuel, c2_nofuel, c3_nofuel, c4_nofuel, c5_nofuel, c6_nofuel, c7_nofuel, c8_nofuel, bcc_fill]
ray_plot2.filename = '3d-bcc-cell.png'
ray_plots2 = openmc.Plots([ray_plot2])
ray_plots2.export_to_xml()
openmc.plot_geometry()

In [ ]:
openmc.run(particles = 25000, geometry_debug=True)